# Describe a PIV Result File

In this example, we want to describe the result file of the ILA Vortex pair (source: https://www.pivtec.com/download/samples/VortexPairSeq.zip)

Where to start? Let's first evaluate what we have:

General context:
- The general concept of a dataset is described by `dcat:Dataset`
- The file is described by `pivmeta:PivResultDistribution` and is part of the `dcat:Dataset`

Specific information:<br>
Of greater interest is the PIV process including the PIV parameters leading to the dataset. A `dcat:Dataset` is the output of the PIV process. For this, we can use [`m4i:output of`](http://purl.obolibrary.org/obo/RO_0002353):

In [1]:
from pivmetalib import pivmeta, dcat, m4i, schema, sd, prov

In [2]:
result_dist = pivmeta.PivResultDistribution(
    title='Result File',
    downloadURL='file:///vp1a.dat'
)
# the "downloaded" file must exist:
assert result_dist.download().exists()

In [3]:
result_dist.downloadURL

Url('file:///vp1a.dat')

In [4]:
pivmeta.PivImageType.ExperimentalImage

<PivImageType.ExperimentalImage: 'https://matthiasprobst.github.io/pivmeta#ExperimentalImage'>

In [5]:
piv_image_dist = pivmeta.PivImageDistribution(
    title='ILA Vortex Pair Images',
    download_URL='https://www.pivtec.com/download/samples/pivimg1.zip',
    piv_image_type=pivmeta.PivImageType.ExperimentalImage
)
piv_image_dist.piv_image_type

<PivImageType.ExperimentalImage: 'https://matthiasprobst.github.io/pivmeta#ExperimentalImage'>

In [6]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=[result_dist, piv_image_dist]
)
ds

Dataset(title=ILA Vortex Pair, distribution=[PivResultDistribution(title=Result File, downloadURL=file:///vp1a.dat), PivImageDistribution(title=ILA Vortex Pair Images, download_URL=https://www.pivtec.com/download/samples/pivimg1.zip, piv_image_type=PivImageType.ExperimentalImage)])

Before defining the PIV processing steps, we need to describe the software used:

In [7]:
piv_software = pivmeta.PIVSoftware(
    author=prov.Organisation(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)

In [8]:
# import h5py
# from typing import Dict


# import h5rdmtoolbox as h5tbx
# h5tbx.set_config(natural_naming=False)

# with h5tbx.File() as h5:
#     piv_software.dump_hdf(h5)
#     h5.dump()
#
# print(piv_software.dump_jsonld())

## Processing steps

A `PivProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [9]:
from ontolutils import PIVMETA

In [10]:
pre = pivmeta.PivPreProcessing(
    name='Image pre processing',
    realizesMethod=[
        m4i.Method(
            description='Rotates the input image by 180 deg',
            hasParameter=m4i.NumericalVariable(
                name='rotation',
                hasNumericalValue=180,
                hasUnit='deg',
                kindOfQuantity='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        m4i.Method(
            description='Applies a median filter to the image',
            hasParameter=pivmeta.NumericalVariable(
                hasStandardName=PIVMETA.image_filter_kernel_size,
                hasNumericalValue=3,
            )
        )
    ]
)
pre

PivPreProcessing(realizesMethod=[Method(description=Rotates the input image by 180 deg, hasParameter=NumericalVariable(hasNumericalValue=180, hasUnit=deg, name=rotation, kindOfQuantity=https://qudt.org/vocab/unit/DEG)), Method(description=Applies a median filter to the image, hasParameter=NumericalVariable(hasNumericalValue=3, hasStandardName=https://matthiasprobst.github.io/pivmeta#image_filter_kernel_size))], name=Image pre processing)

## 2. PIV evaluation

The PIV evaluation is a `m4i:ProcessingStep`. It realizes the methods
- `pivemta:CorrelationMethod` (e.g. FFT, ... ),
- `pivemta:InterrogationMethod` (e.g. `pivemta:Multigrid`, ...) and
- `pivemta:PeakSearchMethod`.

The outlier detection (validation) is a sub-processing step (`pivemta:PivValidation`), because it can realize multiple methods. Most popular ones are described by classes:
- `pivemta:DynMean`.

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [11]:
calgo = pivmeta.CorrelationAlgorithm(
    name='Standard (FFT) Correlation',
    description='The default mode of cross-correlation using FFTs to speed the computation. ' 
    'In principle the sum of pixel-wise multiplication of intensities is computed for each ' 
    'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
calgo

CorrelationAlgorithm(description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)., name=Standard (FFT) Correlation)

### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [12]:
v = pivmeta.NumericalVariable(
            hasStandardName=PIVMETA.x_initial_interrogation_window_size,
            hasNumericalValue=64,
        )
v

NumericalVariable(hasNumericalValue=64, hasStandardName=https://matthiasprobst.github.io/pivmeta#x_initial_interrogation_window_size)

In [13]:
#print(v.dump_jsonld())

In [14]:
int_meth = m4i.Method(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    hasParameter=[
        pivmeta.NumericalVariable(
            hasStandardName=PIVMETA.x_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        pivmeta.NumericalVariable(
            hasStandardName=PIVMETA.y_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        pivmeta.NumericalVariable(
            hasStandardName=PIVMETA.x_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        pivmeta.NumericalVariable(
            hasStandardName=PIVMETA.y_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        pivmeta.NumericalVariable(
            hasStandardName=PIVMETA.x_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        pivmeta.NumericalVariable(
            hasStandardName=PIVMETA.y_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        pivmeta.NumericalVariable(
            # hasStandardName=PIVMETA.number_of_multigrid_passes,
            hasVariableDescription='Number of multigrid passes',
            hasNumericalValue=3,
        )
    ]
)
int_meth.hasParameter[-1]

NumericalVariable(hasVariableDescription=Number of multigrid passes, hasNumericalValue=3)

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [15]:
median_test = pivmeta.OutlierDetectionMethod(
    name='normalized median test',
    hasParameter=m4i.NumericalVariable(
        label='threshold',
        hasNumericalValue=3.0
    )
)

In [16]:
dyn_mean = pivmeta.OutlierDetectionMethod(
    name='dynamic mean test',
    hasParameter=[
        m4i.NumericalVariable(
            label='mean',
            hasNumericalValue=2.0
        ),
        m4i.NumericalVariable(
            label='var',
            hasNumericalValue=1.0
        )
    ]
)

In [17]:
proc = pivmeta.PivEvaluation(
    label='piv evaluation',
    realizesMethod=[
        calgo,
        int_meth,
        median_test,
        dyn_mean
    ]
)
proc

PivEvaluation(label=piv evaluation, realizesMethod=[CorrelationAlgorithm(description=The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)., name=Standard (FFT) Correlation), Method(description=Run a multigrid PIV algorithm on all images, hasParameter=[NumericalVariable(hasNumericalValue=64, hasStandardName=https://matthiasprobst.github.io/pivmeta#x_initial_interrogation_window_size), NumericalVariable(hasNumericalValue=64, hasStandardName=https://matthiasprobst.github.io/pivmeta#y_initial_interrogation_window_size), NumericalVariable(hasNumericalValue=16, hasStandardName=https://matthiasprobst.github.io/pivmeta#x_final_interrogation_window_size), NumericalVariable(hasNumericalValue=16, hasStandardName=https://matthiasprobst.github.io/pivmeta#y_final_interrogation_window_size), NumericalVariable(hasNumericalValue=8, hasStandardName=https://matthiasprobst.github.io/pivmeta#x_final_interrogation_window_overlap_size), NumericalVariable(hasNumericalValue=8, hasStandardName=https://matthiasprobst.github.io/pivmeta#y_final_interrogation_window_overlap_size), NumericalVariable(hasVariableDescription=Number of multigrid passes, hasNumericalValue=3)], name=Multigrid interrogation method), OutlierDetectionMethod(hasParameter=NumericalVariable(label=threshold, hasNumericalValue=3.0), name=normalized median test), OutlierDetectionMethod(hasParameter=[NumericalVariable(label=mean, hasNumericalValue=2.0), NumericalVariable(label=var, hasNumericalValue=1.0)], name=dynamic mean test)])

In [21]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    hasParameter=m4i.NumericalVariable(label='kernel', hasNumericalValue=2.0)
)

In [22]:
post = pivmeta.PivPostProcessing(
    label='Post processing',
    realizesMethod=data_smoothing
)
            

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [24]:
post.hasOutput = ds

## 4. dump piv run to JSON-LD

In [26]:
piv = m4i.ProcessingStep(
    label='PIV Run',
    startsWith=pre,
    endsWith=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = piv_software
proc.has_employed_tool = piv_software
post.has_employed_tool = piv_software

In [27]:
with open('piv_process.json', 'w') as f:
    f.write(proc.dump_jsonld())

In [34]:
print(piv.dump_jsonld(context="https://raw.githubusercontent.com/matthiasprobst/pivmeta/main/pivmeta_context.jsonld"))

{
    "@context": {
        "@import": "https://raw.githubusercontent.com/matthiasprobst/pivmeta/main/pivmeta_context.jsonld",
        "local": "https://local-domain.org/"
    },
    "@graph": [
        {
            "@id": "local:86014695-fe85-451e-b78c-2ecc087f6106",
            "@type": "piv post processing",
            "has_employed_tool": {
                "@id": "local:67046a17-5c3d-429a-adef-434acc980a97"
            },
            "has_output": {
                "@id": "local:d964e055-62bb-430d-91fc-887b95a4dd63"
            },
            "m4i:hasOutput": {
                "@id": "local:0cd5c7dd-b7a9-4b31-9eb8-a4582c796a30"
            },
            "m4i:realizesMethod": {
                "@id": "local:0d931bfb-8db9-407a-9ab0-e8fedb039db3"
            },
            "rdfs:label": "Post processing"
        },
        {
            "@id": "local:f29184a1-b5fd-4bae-a89d-5b8276fd2351",
            "@type": "numerische Variable",
            "hat Zahlenwert": "3.0",
            "

In [35]:
# print(dump_jsonld([pre, proc]))